In [12]:
import numpy as np
import matplotlib.pyplot as plt
from medpy import io
from radiomics import featureextractor, imageoperations
import six
import os
import pandas as pd
import SimpleITK as sitk
import seaborn as sns
from sklearn.metrics import roc_auc_score

import math

def get_image_and_segmentaion(patient_id, study_id, modality = "t2w"):
    if not isinstance(patient_id, str):
        patient_id = str(patient_id)

    if not isinstance(study_id, str):
        study_id = str(study_id)
    
    image = None
    segmentation = None
    for file in os.listdir(data_path + patient_id + "/"):
        string = "{0}_{1}_{2}.mha".format(patient_id,study_id,modality)
        if string == file:
            if study_id in file:
                image = sitk.ReadImage(data_path + patient_id + "/" +  file, sitk.sitkFloat32)
                path_image = data_path + patient_id + "/" +  file

    for file in os.listdir(seg_guer_path):
        if patient_id+"_"+study_id + ".nii.gz" == file:
            
            segmentation = sitk.ReadImage(seg_guer_path + file, sitk.sitkUInt8)
            path_mask = seg_guer_path + file

    return (image, segmentation)

def plot_sitk(sitk_obj, mask=None):

    pixel_array = sitk.GetArrayFromImage(sitk_obj)
    
    columns = 5
    rows = math.ceil(pixel_array.shape[0] / columns)

    fig = plt.figure(figsize=(20,rows*4))

    for i in range(pixel_array.shape[0]):
        #im = data[:,:,i]
        #mask = 
        fig.add_subplot(rows, columns, i+1)
        plt.imshow(pixel_array[i], cmap="gray", interpolation="none")
        if mask is not None:
            array = sitk.GetArrayFromImage(mask)
            plt.imshow(array[i], cmap="Reds", alpha = 0.9*(array[i]>0), interpolation="none")

data_path = "/data1/practical-sose23/morphometric/data/"
seg_guer_path = "/data1/practical-sose23/morphometric/picai_labels/anatomical_delineations/whole_gland/AI/Guerbet23/"
seg_bosma_path = "/data1/practical-sose23/morphometric/picai_labels/anatomical_delineations/whole_gland/AI/Bosma22b/"

In [19]:
patient = 10010
study = 1000010

t2w_image, mask = get_image_and_segmentaion(patient, study, "t2w")
adc_image, mask = get_image_and_segmentaion(patient, study, "adc")

In [20]:
print(adc_image.GetOrigin())
print(t2w_image.GetOrigin())

(-85.58146266535, -142.07086996919, -28.491073594095)
(-62.611289988486, -109.44163600357, -31.266651149489)


In [21]:
def resample_mask(image, mask):
    resample = sitk.ResampleImageFilter()
    resample.SetInterpolator = sitk.sitkNearestNeighbor

    resample.SetOutputOrigin(image.GetOrigin())
    resample.SetOutputDirection(image.GetDirection())

    resample.SetSize(image.GetSize()) 
    resample.SetOutputSpacing(image.GetSpacing())
    


    resample_mask = resample.Execute(mask)

    return resample_mask
  

In [ ]:
new_mask = resample_mask(adc_image, mask)
plot_sitk(adc_image, new_mask)

In [30]:
seg_guer_path_resample = "/data1/practical-sose23/morphometric/picai_labels/anatomical_delineations/whole_gland/AI/Guerbet23_resampled/"

In [36]:
gt = pd.read_csv("/data1/practical-sose23/morphometric/picai_labels/clinical_information/marksheet.csv")
for index,row in gt.iterrows():
    print(index, end="\r")
    patient = str(row["patient_id"])
    study = str(row["study_id"])
    adc_image, mask = get_image_and_segmentaion(patient, study, "adc")
    new_mask = resample_mask(adc_image, mask)

    writer = sitk.ImageFileWriter()
    writer.SetFileName(seg_guer_path_resample + patient + "_" + study +".nii.gz")
    writer.Execute(new_mask)
